# Search with base models

## Goal

Can we solve ARC tasks using base models with access to a DSL?

## Imports

In [ ]:
import os
import logging
from arc25.utils import get_least_used_gpu_index
from arc25.logging import configure_logging, log_execution_time

configure_logging()
os.environ['CUDA_VISIBLE_DEVICES'] = str(get_least_used_gpu_index())

# Add VLLM specific environment variables to avoid common issues
os.environ['VLLM_USE_MODELSCOPE'] = 'False'
os.environ['VLLM_WORKER_MULTIPROC_METHOD'] = 'spawn'

In [ ]:
import time
import importlib
import inspect
import json
import gc
import random
import pandas as pd
from tqdm.auto import tqdm

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import PeftModel
from vllm import LLM, SamplingParams
from vllm.sampling_params import BeamSearchParams

from arc25.training_tasks import *
from arc25.encoders import create_grid_encoder
from arc25.prompting import pretty_print_prompt, Template
from arc25.metrics import pixel_similarity_score, correct_grids_score
import arc25.BARC_dsl as dsl

## Code

### Prompt

https://github.com/flowersteam/SOAR/blob/main/soar/prompt.py

In [ ]:
def extract_footprint(module_name: str, show_types: bool = False) -> str:
    """
    Load a module by name, then return a newline-separated list of all
    top-level functions in it, in the form:

      def func_name(arg1, arg2) -> return

    If show_types=True, annotations are included; otherwise only names.
    """
    mod = importlib.import_module(module_name)
    footprints = []

    for name, fn in inspect.getmembers(mod, inspect.isfunction):
        # skip imports from elsewhere
        if fn.__module__ != module_name or name.startswith("_"):
            continue

        sig = inspect.signature(fn)
        if not show_types:
            # strip type info
            params = [p.name for p in sig.parameters.values()]
            sig_text = f"({', '.join(params)})"
        else:
            sig_text = str(sig)

        footprints.append(f"- dsl.{name}{sig_text}")

    return "\n".join(footprints)

print(extract_footprint('arc25.BARC_dsl', show_types=True))

In [ ]:
with open('/mnt/hdd0/Kaggle/arc25/data/arc-prize-2024/arc-agi_training_challenges.json', 'r') as f:
    training_challenges = json.load(f)

def get_task(task_name):
    if task_name in training_challenges:
        task_data = training_challenges[task_name]
        inputs = [Img(sample['input']) for sample in task_data['train']]
        outputs = [Img(sample['output']) for sample in task_data['train']]
        return Task(inputs=inputs, outputs=outputs, code='', name=task_name)
    raise ValueError(f"Task {task_name} not found in training challenges.")

In [ ]:
system_prompt = """You are an advanced AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks."""

prompt_template_text ="""You are tasked with solving a transformation problem from the Abstraction and Reasoning Challenge (ARC).
Implement the transformation rules as a Python function.
You should only write the implemented the transformation in code.
You must write code in triple backticks (```python and then ). You must write a function called `transform` which takes a single argument, the input grid as `list[list[int]]`, and returns the transformed grid (also as `list[list[int]]`).

## Key Priors:

- **Objectness**: Consider the grid as containing objects (groups of connected cells) rather than just individual pixels.
- **Goal-Directed**: The transformation should achieve a specific goal, such as creating symmetry or changing the color of specific objects.
- **Numbers & Counting**: Keep track of the number of objects, sizes, and their relative positions.
- **Geometry & Topology**: Use spatial relationships such as adjacency, enclosure, or symmetry.

Carefully analyze the examples and find the underlying transformation logic.

## Domain Specific Primitive Functions

You can use the already implemented following functions to manipulate the grid:

{{ dsl }}

The dsl has been already imported, so just simply call the functions as needed. F.e. dsl.foo()
Do not import the dsl again, just use it directly.

## Examples

Below are several input-output examples that illustrate the transformation.
Your function should generalize the pattern from these examples to solve any input following the same logic.

{% for sample in train_samples %}
### Example {{ loop.index }}

#### Input

{{ sample.input }}

#### Output

{{ sample.output }}
{% endfor %}
"""

prompt_template = Template(prompt_template_text)


def create_prompt_from_task(task, grid_encoder, tokenizer, shuffle_train_samples=False, remove_last_train_sample=False):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    if shuffle_train_samples:
        random.shuffle(train_samples)
    if remove_last_train_sample and len(train_samples) > 1:
        train_samples = train_samples[:-1]
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

In [ ]:
# v1. One problem of v1 is that 13% of the tasks have predicted the exact same code. So let's rewrite a prompt to avoid that
prompt_novel_tasks_text_v1 = prompt_template_text + """
## Already generated code

Below are python functions already generated to try to solve the task. Take them into account when generating the new code.
Your code should be different from the already generated code, it should generate a new and different solution to the task.
The functions are given to avoid repeating the same code. Your code should be a new and different solution to the task.

{% for prediction in previous_predictions %}
### Code {{ loop.index }}

python
{{ prediction }}
```

{% endfor %}
"""

# v2, 8% exact repetitions
prompt_novel_tasks_text_v2 = prompt_template_text + """
## Already generated code

Below are python functions already generated to try to solve the task. 
Do not repeat them, make sure to generate a new and different solution to the task.

{% for prediction in previous_predictions %}
### Code sample{{ loop.index }}

```python
{{ prediction }}
```

{% endfor %}

## New solution

Now implement a new and original solution to the task.
"""

# v3, 5% of repeated code
prompt_novel_tasks_text_v3 = prompt_template_text + """
## Already generated code

Below are python functions already generated to try to solve the task. 
Do not repeat them, make sure to generate a new and different solution to the task.

{% for prediction in previous_predictions %}
### Code sample{{ loop.index }}

DO NOT REPEAT THIS CODE, generate a new and different solution to the task.

```python
{{ prediction }}


{% endfor %}

## New solution

Now implement a new and original solution to the task.
Write some code that solves the task using a different approach than the previous ones.
"""

# system_prompt = """You are an advanced AI assistant specialized in solving Abstract Reasoning Corpus (ARC-AGI) tasks.
# Do not repeat any of the code snippets provided in the previous predictions. Always generate a new and different solution to the task."""

prompt_novel_tasks_template = Template(prompt_novel_tasks_text_v1)
# TODO: add scores to the prompt
#Pixel similarity scores: {{ prediction.pixel_similarity_scores }}
#Correct grids scores: {{ prediction.correct_grids_scores }}

def create_prompt_for_novel_solutions(task, grid_encoder, tokenizer, previous_predictions):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True),
                         previous_predictions=previous_predictions)
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_novel_tasks_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

In [ ]:
refine_prompt_v1 = prompt_template_text + """
## Code to be refined

Below are python functions already generated to try to solve the task.
Those functions do not solve the task, but the direction of the solution is correct.
Refine them into a new function that solves the task.

{% for prediction in previous_predictions %}
### Code sample {{ loop.index }}

```python
{{ prediction }}
```

{% endfor %}

## Refined solution

Now implement a new function that refines the previous code and solves the task.
"""

refine_prompt_template = Template(refine_prompt_v1)


def create_refine_prompt(task, grid_encoder, tokenizer, previous_predictions):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True),
                         previous_predictions=previous_predictions)
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": refine_prompt_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

In [ ]:
prompt_variations = [
"""You are tasked with solving a transformation problem from the Abstraction and Reasoning Challenge (ARC).
Implement the transformation rules as a Python function.
You should only write the implemented the transformation in code.
You must write code in triple backticks (```python and then ```). You must write a function called `transform` which takes a single argument, the input grid as `list[list[int]]`, and returns the transformed grid (also as `list[list[int]]`).

## Key Priors:

- **Objectness**: Consider the grid as containing objects (groups of connected cells) rather than just individual pixels.
- **Goal-Directed**: The transformation should achieve a specific goal, such as creating symmetry or changing the color of specific objects.
- **Numbers & Counting**: Keep track of the number of objects, sizes, and their relative positions.
- **Geometry & Topology**: Use spatial relationships such as adjacency, enclosure, or symmetry.

Carefully analyze the examples and find the underlying transformation logic.

## Domain Specific Primitive Functions

You can use the already implemented following functions to manipulate the grid:

{{ dsl }}

The dsl has been already imported, so just simply call the functions as needed. F.e. dsl.foo()
Do not import the dsl again, just use it directly.

## Examples

Below are several input-output examples that illustrate the transformation.
Your function should generalize the pattern from these examples to solve any input following the same logic.

{% for sample in train_samples %}
### Example {{ loop.index }}

#### Input

{{ sample.input }}

#### Output

{{ sample.output }}
{% endfor %}
""",
"""You are an expert ARC solver. Your goal is to deduce the underlying transformation logic from the provided examples and implement it in a Python function.

## Instructions

Your response must have two parts:

1.  **Analysis**: First, explain your reasoning. Describe the pattern you've identified, the key observations, and the step-by-step logic for the transformation.
2.  **Implementation**: Second, provide the Python code.

---

### 1. Analysis and Reasoning 🧠

Carefully observe the input-output pairs. Based on your analysis, describe the core transformation logic. Answer questions like:

* What objects or patterns are present in the inputs?
* How do these objects or patterns change to produce the outputs?
* What is the consistent rule that applies to all examples?
* What is your step-by-step plan to implement this rule?

---

### 2. Python Implementation 💡

Now, implement the logic you described above.

* Wrap your code in a single Python block (```python ... ```).
* Create a function `transform(grid: list[list[int]]) -> list[list[int]]`.
* Do not write any code outside of this function.

---

### Guiding Principles & Resources

To help you, keep these concepts in mind and use the provided functions.

* **Core Concepts**: Look for principles like object detection, counting, symmetry, rotation, color manipulation, and geometric relationships (e.g., containment, adjacency).
* **Domain-Specific Functions (DSL)**: You have access to a pre-loaded library of helper functions called `dsl`. Use them directly (e.g., `dsl.some_function()`) without importing them.

{{ dsl }}

---

### Training Examples

These examples demonstrate the transformation rule you must implement.

{% for sample in train_samples %}
#### Example {{ loop.index }}

**Input:**

{{ sample.input }}

**Output:**

{{ sample.output }}

{% endfor %}
""",
"""You are a research AI specializing in abstract pattern recognition. Your task is to solve the provided ARC puzzle by first deducing the transformation rule and then implementing it.

Your final output should be a single Python code block containing the solution. Before you write the code, you must outline your thinking process.

---

### **Part 1: Formulate Your Hypothesis**

Analyze the training examples to develop a theory of the transformation. Please structure your analysis as follows:

1.  **General Observations:**
    * Briefly describe the common elements and changes across all `input` -> `output` pairs. Consider colors, shapes, sizes, counts, and spatial arrangements.

2.  **Core Hypothesis:**
    * In one or two sentences, state the specific rule that you believe governs the transformation. This rule must be general enough to work for all examples.

3.  **Algorithm Outline:**
    * Provide a high-level, step-by-step plan or pseudocode that describes how to implement your hypothesis. This should be a clear blueprint for your code.

---

### **Part 2: Implement Your Solution**

Translate your algorithm into a Python function.

* You must define a function: `transform(grid: list[list[int]]) -> list[list[int]]`.
* Enclose your complete solution in a single ` ```python ` code block.
* Do not include any other text or explanation outside the code block in this final part.

---

### **Reference Materials**

* **Available Tools (`dsl`)**: You can use the functions from the `dsl` library provided below. They are already imported for your use. Just call them as `dsl.function_name()`.

{{ dsl }}

* **Case Studies (Training Data)**: Use these input-output pairs to derive your hypothesis.

{% for sample in train_samples %}
**Case Study {{ loop.index }}**

*Input Grid:*

{{ sample.input }}



*Resulting Grid:*


{{ sample.output }}


{% endfor %}""",
"""You are tasked with solving a transformation problem from the Abstraction and Reasoning Challenge (ARC).

Implement the transformation logic as a Python function called `transform`.

Only output the code. The code must be enclosed in triple backticks like this:

```python
def transform(grid: list[list[int]]) -> list[list[int]]:
    ...
````

Your function should take a single argument, `grid`, and return a transformed grid of the same type.

## Key Priors

Keep in mind the following principles to guide your implementation:

* **Objectness**: Treat the grid as composed of objects—groups of connected cells—rather than individual pixels.
* **Goal-Directedness**: Assume the transformation pursues a goal, such as restoring symmetry, recoloring specific objects, or rearranging structures.
* **Counting & Structure**: Pay attention to the number, size, and spatial arrangement of objects.
* **Geometry & Topology**: Consider relationships like adjacency, symmetry, containment, alignment, or enclosure.

Analyze the examples carefully to uncover the general transformation logic.

## Available Tools

You may use the following domain-specific primitive functions, already available in the `dsl` module:

{{ dsl }}

These have been imported for you. Call them directly as needed, e.g., `dsl.some_function(...)`.

Do **not** re-import the module.

## Examples

Study the input-output examples below. Your function should generalize from these to handle unseen inputs that follow the same transformation rule.

{% for sample in train_samples %}

### Example {{ loop.index }}

#### Input

{{ sample.input }}

#### Output

{{ sample.output }}

{% endfor %}
""",
"""You are a reasoning agent solving an ARC (Abstraction and Reasoning Corpus) task.

Your job is to discover the transformation rule underlying several input-output grid pairs, and implement this rule in Python.

## Instructions

Write a function named `transform(grid: list[list[int]]) -> list[list[int]]` that implements the inferred rule.

- Output **only the code**, inside triple backticks like so:

```python
def transform(grid: list[list[int]]) -> list[list[int]]:
    ...
````

* The code should work for any input grid that follows the same transformation logic.

## Reasoning Heuristics

Use the following principles to guide your analysis:

* **Objects, not pixels**: Identify groups of connected cells that behave as coherent entities.
* **Functional transformation**: The output is not arbitrary—it results from applying a consistent operation or goal to the input.
* **Count, compare, align**: Track quantities, positions, symmetries, or relative motion between elements.
* **Local vs global**: Consider whether the rule acts on each object individually or on the grid as a whole.

## Tools

You are allowed to use predefined helper functions provided via the `dsl` module:

{{ dsl }}

These are already available in the environment. Call them directly as needed, e.g., `dsl.some_function(...)`.

Do **not** re-import or redefine the `dsl`.

## Examples

Analyze the following training examples to understand the transformation. Your function should generalize the rule that maps each input to its corresponding output.

{% for sample in train_samples %}

### Example {{ loop.index }}

#### Input

{{ sample.input }}

#### Output

{{ sample.output }}

{% endfor %}
""",
"""You are an expert AI programmer specializing in solving Abstraction and Reasoning Challenge (ARC) problems.

Your goal is to deduce the transformation logic from the provided examples and implement it in a single Python function.

## 1. Analyze the Examples

Carefully study the input-output pairs below. Identify the underlying pattern, rule, or transformation that connects each input grid to its corresponding output grid.

{% for sample in train_samples %}
### Example {{ loop.index }}
**Input:**


{{ sample.input }}


**Output:**


{{ sample.output }}


{% endfor %}

---

## 2. Key Concepts for Reasoning

As you analyze, consider the following principles to uncover the solution:
- **Object-Based Analysis:** Treat connected groups of same-colored pixels as distinct objects. Analyze their properties (size, shape, color) and relationships (position, containment, adjacency).
- **Symmetry & Repetition:** Look for patterns involving symmetry (reflection, rotation), repetition, or fractals.
- **Counting & Logic:** The solution may depend on the number of objects, the count of pixels of a certain color, or other numerical properties.
- **Geometric Transformations:** Think about transformations like moving, scaling, rotating, cropping, or copying objects.

---

## 3. Available Tools

You have access to a pre-loaded library of helper functions available through the `dsl` object. You can and should use these functions to implement the transformation logic.

**Available `dsl` functions:**


{{ dsl }}


**Note:** The `dsl` is already imported. Call functions directly, for example: `dsl.some_function(...)`.

---

## 4. Implementation

Now, write the Python function that solves the task.

### Requirements:
- The function must be named `transform`.
- It must accept one argument: `grid` (`list[list[int]]`).
- It must return the transformed grid (`list[list[int]]`).
- **Your entire response must be a single Python code block.** Do not write any explanations, comments, or text outside of the ```python ... ``` block.
""",
"""You are solving a transformation task from the Abstraction and Reasoning Challenge (ARC).

Write a Python function to implement the transformation. Provide only the function code, wrapped in triple backticks:

```python
def transform(grid: list[list[int]]) -> list[list[int]]:
    # your implementation here
````

### Key Principles

* **Objectness**: Treat connected cells as objects, not isolated pixels.
* **Goal‑Directed**: Aim for a specific outcome (e.g., symmetry, color change).
* **Counting**: Track object counts, sizes, and positions.
* **Spatial Reasoning**: Leverage adjacency, enclosure, and symmetry.

Analyze the examples to infer the underlying rule.

### Available Primitives

You may call any of these pre‑imported DSL functions directly (e.g., `dsl.foo()`):

{{ dsl }}

> **Do not** import the DSL module again.

### Examples

Use the following I/O pairs to generalize the rule. Your `transform` must handle any input that follows the same pattern.

{% for sample in train_samples %}

#### Example {{ loop.index }}

**Input**


{{ sample.input }}


**Output**


{{ sample.output }}


{% endfor %}
""",
"""
You are solving an Abstraction and Reasoning Challenge (ARC) transformation task.

Implement the grid transformation as a Python function.

**Requirements**  
- Provide only the code for your transformation.  
- Enclose your code in triple backticks with a Python syntax hint:  
  ```python  
  # your solution  
````

* Define a function `transform(input_grid: list[list[int]]) -> list[list[int]]`
  that returns the correctly transformed grid.

## Key Priors

1. **Objectness**
   Treat contiguous regions of the same color as objects, not isolated pixels.
2. **Goal-Directed**
   Identify and apply the specific transformation goal (e.g., symmetry, color-change).
3. **Counting & Numbers**
   Count objects, compare sizes, and use relative positioning.
4. **Geometry & Topology**
   Leverage spatial relationships (adjacency, enclosure, symmetry).

Analyze the examples to infer the underlying rule and generalize it.

## Domain-Specific Primitives

You may call any of the imported DSL functions directly.
Example usage:

```python
dsl.foo(...)
```

*Do not* re-import the DSL module.

{{ dsl }}

## Examples

Below are training examples illustrating the desired transformation. Your solution should generalize from these.

{% for sample in train_samples %}

### Example {{ loop.index }}

**Input**


{{ sample.input }}


**Output**


{{ sample.output }}


{% endfor %}
"""
]
len(prompt_variations)
def create_prompt_variations_from_task(idx, task, grid_encoder, tokenizer, shuffle_train_samples=False):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    if shuffle_train_samples:
        random.shuffle(train_samples)
    prompt_template = Template(prompt_variations[idx])
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

In [ ]:
def create_random_suggestions_for_prompt(module_name: str = 'arc25.BARC_dsl') -> str:
    mod = importlib.import_module(module_name)
    func_names = []
    for name, fn in inspect.getmembers(mod, inspect.isfunction):
        if fn.__module__ != module_name or name.startswith("_"):
            continue
        func_names.append(name)
    selection = random.sample(func_names, min(5, len(func_names)))
    suggestion = '\n\n**Bonus**: You will get bonus points if you use any of the following dsl functions in your solution:\n\n'
    suggestion += '\n'.join(f"- dsl.{name}" for name in selection)
    return suggestion


def create_prompt_with_dsl_suggestions(task, grid_encoder, tokenizer, shuffle_train_samples=False):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    if shuffle_train_samples:
        random.shuffle(train_samples)
    prompt_template = Template(prompt_template_text + create_random_suggestions_for_prompt())
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

### Model

In [ ]:
@log_execution_time
def load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1, max_model_len=32000):
    logging.info(f"Loading model from {model_path}")
    cleanup_gpu()
    llm = LLM(
        model=model_path,
        gpu_memory_utilization=0.9,  # Use less GPU memory
        # max_model_len=4096,  # Limit context length
        trust_remote_code=True,
        dtype="bfloat16",  # Use float16 to save memory
        tensor_parallel_size=tensor_parallel_size,  # Single GPU
        quantization="bitsandbytes" if use_4bit_quantization else None,
        enable_prefix_caching=True, # Seems that it is true by default, but let's be explicit
        max_model_len=max_model_len,
    )
    if model_path.endswith('.gguf'):
        tokenizer_path = os.path.join(os.path.dirname(model_path), 'tokenizer')
    else:
        tokenizer_path = model_path
    tokenizer = AutoTokenizer.from_pretrained(tokenizer_path)
    return llm, tokenizer


def cleanup_gpu():
    """Clean up GPU memory before loading VLLM"""
    gc.collect()
    torch.cuda.empty_cache()
    if torch.cuda.is_available():
        torch.cuda.synchronize()

### Code

In [ ]:
def parse_python_code(text):
    # Extract Python code from the text
    if '```python' not in text:
        return ''
    code = text.split('```python')[1]
    if not '```' in code:
        return ''

    code = code.split('```')[0].strip()
    return code

In [ ]:
def curate_python_code(code):
    remove_line_keywords = ['import dsl', 'from dsl import ', 'print(']
    code = '\n'.join(line for line in code.split('\n') if not any(keyword in line for keyword in remove_line_keywords))
    return code.strip()

def add_additional_imports(code):
    additional_imports = [
        'from typing import List, Tuple',
        'import numpy as np',
        'import numpy'
    ]
    imports = '\n'.join(additional_imports)
    return imports + '\n' + code if code else imports

### Validations

In [ ]:
def validate_outputs(outputs):
    if not outputs:
        raise ValueError("Outputs list is empty")
    return [_validate_output(output) for output in outputs]

def _validate_output(output):
    if output is None:
        raise ValueError("Output is None")
    output = np.array(output) # otherwise I see weird outputs that mix list and numpy arrays
    if output.ndim != 2:
        raise ValueError(f"Output is not a 2D array. Output shape: {output.shape}")
    if max(output.shape) > 35:
        raise ValueError(f"Output is too large, the maximum allowed shape is 30x30. Output shape: {output.shape}")
    if min(output.shape) == 0:
        raise ValueError(f"Output has zero dimension, it is empty. Output shape: {output.shape}")
    if np.max(output) > 9 or np.min(output) < 0:
        raise ValueError(f"Output contains invalid values, expected values in range [0, 9]. Output max: {np.max(output)}, min: {np.min(output)}")
    return output

In [ ]:
import hashlib

def fingerprint(prediction):
    """
    Create a compact hash for a list of matrices.
    Includes shape & dtype to distinguish e.g. (2×2) from (4×1).
    """
    h = hashlib.sha256()
    for m in prediction:
        # incorporate shape and dtype in a reproducible way
        h.update(str(m.shape).encode())
        h.update(m.dtype.str.encode())
        # raw data bytes
        h.update(m.tobytes())
    return h.hexdigest()

### Metrics

In [ ]:
def compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds):
    df = pd.DataFrame(columns=['valid code', 'valid outputs', 'unique outputs', 'dsl usage', 'pixel similarity', 'correct grids', 'solved task'])
    for task_id in task_ids:
        df.loc[task_id, 'valid code'] = len(predicted_code[task_id])/n_preds
        df.loc[task_id, 'valid outputs'] = len(predicted_outputs[task_id])/n_preds
        df.loc[task_id, 'unique outputs'] = len(set(fingerprint(output) for output in predicted_outputs[task_id]))/n_preds
        df.loc[task_id, 'dsl usage'] = sum(1 for code in predicted_code[task_id] if 'dsl.' in code)/n_preds

        task = get_task(task_id)
        task_predicted_outputs = predicted_outputs[task_id]
        scores = sorted([np.mean([pixel_similarity_score(output, pred) for output, pred in zip(task.outputs, predictions)]) for predictions in task_predicted_outputs])
        df.loc[task_id, 'pixel similarity'] = np.mean(scores) if scores else 0.0

        task_outputs = [np.array(output) for output in task.outputs]
        scores = sorted([correct_grids_score(task_outputs, predictions) for predictions in task_predicted_outputs])
        df.loc[task_id, 'correct grids'] = np.mean(scores) if scores else 0.0
        df.loc[task_id, 'solved task'] = int(np.max(scores) == 1) if scores else 0

    df.loc['MEAN'] = df.mean(axis=0)
    return df

In [ ]:
raise

## Independent search

In [ ]:
model_path = "/home/gbarbadillo/models/Qwen2.5-Coder-7B-Instruct"
llm, tokenizer = load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1)

# model_path = "/home/gbarbadillo/models/Qwen3-4B"
# llm, tokenizer = load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1)

# model_path = '/home/gbarbadillo/models/Qwen2.5-Coder-14B-Instruct-GGUF/qwen2.5-coder-14b-instruct-q4_k_m.gguf' # Needs 2 GPUs
# llm, tokenizer = load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=2, max_model_len=16000)

In [ ]:
task_ids = list(training_challenges.keys())
sampling_params = SamplingParams(n=8, temperature=1.0, top_p=0.95, max_tokens=2048)
grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))')
prompts = [create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer) for task_id in task_ids]
if 'Qwen3' in model_path:
    # disable thinking without using the chat template
    prompts = [prompt + '<think>\n\n</think>\n\n' for prompt in prompts]

t0 = time.time()
outputs = llm.generate(prompts, sampling_params)
total_tokens = sum(sum(len(_output.token_ids) for _output in output.outputs) for output in outputs)
inference_time = time.time() - t0
print(f"Total tokens generated: {total_tokens}")
print(f"Time taken: {inference_time:.2f} seconds")
print(f"Average time per task: {inference_time / len(outputs):.2f} seconds")
print(f"Average tokens per task: {total_tokens / len(outputs) / sampling_params.n:.2f} tokens")
print(f"Average tokens per second: {total_tokens / inference_time:.2f} tokens/second")

In [ ]:
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            code = curate_python_code(code)
            predicted_code[task_id].append(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

In [ ]:
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, sampling_params.n)
df

In [ ]:
df.iloc[-1:]

In [ ]:
output_path = f'{os.path.basename(model_path)}_{len(task_ids)}tasks_{sampling_params.n}preds_{int(inference_time)}runtime.csv'
df.to_csv(output_path, index_label='task_id')
print(f"Results saved to {output_path}")

## Sequential search

Let's give the model the context of all the functions generated so far to try induce more diversity.

### Prompt tuning

#### Configuration

In [ ]:
model_path = "/home/gbarbadillo/models/Qwen2.5-Coder-7B-Instruct"
llm, tokenizer = load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1)

In [ ]:
task_ids = list(training_challenges.keys())
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))')

#### Initialization

As a first step let's make sure that we have a single prediction that predicts a valid output for each task

In [ ]:
# first prediction initialization
while True:
    prompts, epoch_task_ids = [], []
    for task_id in task_ids:
        if not predicted_code[task_id]:  # only create prompt if no code has been predicted yet
            prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer)
            prompts.append(prompt)
            epoch_task_ids.append(task_id)
    if not prompts:
        break

    outputs = llm.generate(prompts, sampling_params)

    for task_id, responses in zip(epoch_task_ids, outputs):
        task = get_task(task_id)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    predicted_code[task_id].append(code)
                    predicted_outputs[task_id].append(task_predicted_outputs)
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

In [ ]:
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, 1)
df

#### Independent search Baseline

In [ ]:
predicted_outputs = {key: values[:1] for key, values in predicted_outputs.items()}
predicted_code = {key: values[:1] for key, values in predicted_code.items()}

prompts = []
for task_id in task_ids:
    prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer)
    prompts.append(prompt)

outputs = llm.generate(prompts, sampling_params)
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, 2)
df

#### Compare with sequential search

In [ ]:
predicted_outputs = {key: values[:1] for key, values in predicted_outputs.items()}
predicted_code = {key: values[:1] for key, values in predicted_code.items()}

sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048, repetition_penalty=1.0)

prompts = []
for task_id in task_ids:
    prompt = create_prompt_for_novel_solutions(get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer, previous_predictions=predicted_code[task_id])
    prompts.append(prompt)

outputs = llm.generate(prompts, sampling_params)
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, 2)
df

In [ ]:
total, equal = 0, 0
repeated_code_task_ids = []
for task_id in task_ids:
    if len(predicted_code[task_id]) > 1:
        total += 1
        if predicted_code[task_id][0] == predicted_code[task_id][1]:
            repeated_code_task_ids.append(task_id)
            equal += 1
print(f"Total tasks with multiple predictions: {total}")
print(f"Tasks with exactly equal predictions: {equal} ({equal/total:.2%})")
print(f"Tasks with repeated code: {repeated_code_task_ids}")

In [ ]:
pretty_print_prompt(prompts[task_ids.index(repeated_code_task_ids[0])], default_color='white')

In [ ]:
predicted_code[repeated_code_task_ids[0]]

#### Compare with refine prompt approach

In [ ]:
predicted_outputs = {key: values[:1] for key, values in predicted_outputs.items()}
predicted_code = {key: values[:1] for key, values in predicted_code.items()}

sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048, repetition_penalty=1.0)

prompts = []
for task_id in task_ids:
    prompt = create_refine_prompt(get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer, previous_predictions=predicted_code[task_id])
    prompts.append(prompt)

outputs = llm.generate(prompts, sampling_params)
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, 2)
df

In [ ]:
total, equal = 0, 0
repeated_code_task_ids = []
for task_id in task_ids:
    if len(predicted_code[task_id]) > 1:
        total += 1
        if predicted_code[task_id][0] == predicted_code[task_id][1]:
            repeated_code_task_ids.append(task_id)
            equal += 1
print(f"Total tasks with multiple predictions: {total}")
print(f"Tasks with exactly equal predictions: {equal} ({equal/total:.2%})")
print(f"Tasks with repeated code: {repeated_code_task_ids}")

In [ ]:
pretty_print_prompt(prompts[task_ids.index(repeated_code_task_ids[0])], default_color='white')

In [ ]:
predicted_code[repeated_code_task_ids[0]]

## Increase search diversity

I'm going to make experiments with 8 predictions per task, that should take around 30 minutes per experiment. Hopefully I will have enough resolution to measure changes.

### Default Configuration

In [ ]:
model_path = "/home/gbarbadillo/models/Qwen2.5-Coder-7B-Instruct"
llm, tokenizer = load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1)

In [ ]:
n_preds = 8
task_ids = list(training_challenges.keys())
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))')

### Baseline

In [ ]:
sampling_params = SamplingParams(n=n_preds, temperature=1.0, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
prompts = []
for task_id in task_ids:
    prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder,
                                        tokenizer=tokenizer, shuffle_train_samples=False)
    prompts.append(prompt)

outputs = llm.generate(prompts, sampling_params)
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

### Shuffle train samples

In [ ]:
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
print(f"Generating {n_preds} predictions for {len(task_ids)} tasks...")
for epoch in tqdm(range(n_preds), smoothing=0, desc="Generating predictions"):
    prompts = []
    for task_id in task_ids:
        prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder,
                                         tokenizer=tokenizer, shuffle_train_samples=True)
        prompts.append(prompt)

    outputs = llm.generate(prompts, sampling_params)
    for task_id, responses in zip(task_ids, outputs):
        task = get_task(task_id)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                predicted_code[task_id].append(code)
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    predicted_outputs[task_id].append(task_predicted_outputs)
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

### Effect of temperature

In [ ]:
n_preds = 16
sampling_params = SamplingParams(n=n_preds, temperature=1.2, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
prompts = []
for task_id in task_ids:
    prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder,
                                        tokenizer=tokenizer, shuffle_train_samples=False)
    prompts.append(prompt)

outputs = llm.generate(prompts, sampling_params)
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

### Prompt variations

In [ ]:
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
print(f"Generating {n_preds} predictions for {len(task_ids)} tasks...")
for epoch in tqdm(range(n_preds), smoothing=0, desc="Generating predictions"):
    prompts = []
    for task_id in task_ids:
        prompt = create_prompt_variations_from_task(
            epoch, get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer, shuffle_train_samples=False)
        prompts.append(prompt)
    # pretty_print_prompt(prompts[0], default_color='white')

    outputs = llm.generate(prompts, sampling_params)
    for task_id, responses in zip(task_ids, outputs):
        task = get_task(task_id)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                predicted_code[task_id].append(code)
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    predicted_outputs[task_id].append(task_predicted_outputs)
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

Would this be faster if I give all the prompts at once? I have tested it and no.

### Beam search

In [ ]:
sampling_params = BeamSearchParams(beam_width=n_preds, temperature=1.0, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
prompts = []
for task_id in task_ids:
    prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder,
                                        tokenizer=tokenizer, shuffle_train_samples=False)
    prompts.append(dict(prompt=prompt))

outputs = llm.beam_search(prompts, params=sampling_params)
for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

GPU usage is not constant, average should be around 50%.  
After 8 minutes it has not ended the prediction for just 5 tasks. Making predictions for 400 tasks will take more than 10 hours.
I have stopped it after 11 minutes without response.

### Add suggestion to use some of the DSL functions

In [ ]:
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
print(f"Generating {n_preds} predictions for {len(task_ids)} tasks...")
for epoch in tqdm(range(n_preds), smoothing=0, desc="Generating predictions"):
    prompts = []
    for task_id in task_ids:
        prompt = create_prompt_with_dsl_suggestions(
            get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer, shuffle_train_samples=False)
        prompts.append(prompt)
    # pretty_print_prompt(prompts[0], default_color='white')

    outputs = llm.generate(prompts, sampling_params)
    for task_id, responses in zip(task_ids, outputs):
        task = get_task(task_id)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                predicted_code[task_id].append(code)
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    predicted_outputs[task_id].append(task_predicted_outputs)
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

### Data augmentation

The challenge of data augmentation is that we have to be able to undo the data augmentation to see if the solution was correct or unique.

I'm going to try to find the code of the previous year.

#### Code

In [ ]:
def apply_data_augmentation(task, hflip, n_rot90, color_map=None):
    augmented_task = Task(
        inputs = [geometric_augmentation(grid, hflip, n_rot90) for grid in task.inputs],
        outputs = [geometric_augmentation(grid, hflip, n_rot90) for grid in task.outputs],
        code = '',
        name = task.name,
    )
    if color_map is not None:
        augmented_task = swap_task_colors(augmented_task, color_map)
    return augmented_task


def revert_data_augmentation(grid, hflip, n_rot90, color_map=None):
    grid = revert_geometric_augmentation(grid, hflip, n_rot90)
    if color_map is not None:
        grid = revert_color_swap(grid, color_map)
    return grid


def geometric_augmentation(grid, hflip, n_rot90):
    grid = np.array(grid)
    if hflip:
        grid = np.flip(grid, axis=1)
    grid = np.rot90(grid, k=n_rot90)
    return grid.tolist()


def revert_geometric_augmentation(grid, hflip, n_rot90):
    grid = np.array(grid)
    grid = np.rot90(grid, k=-n_rot90)
    if hflip:
        grid = np.flip(grid, axis=1)
    return grid

def revert_color_swap(grid, color_map):
    reverse_color_map = {v: k for k, v in color_map.items()}
    vectorized_mapping = np.vectorize(reverse_color_map.get)
    return vectorized_mapping(grid)


def swap_task_colors(task, color_map=None, change_background_probability=0.1):
    if color_map is None:
        color_map = get_random_color_map(change_background_probability)
    vectorized_mapping = np.vectorize(color_map.get)
    new_task = Task(
        inputs = [vectorized_mapping(grid) for grid in task.inputs],
        outputs = [vectorized_mapping(grid) for grid in task.outputs],
        code = '',
        name = task.name,)
    return new_task


def get_random_data_augmentation_params():
    params = get_random_geometric_augmentation_params()
    params['color_map'] = get_random_color_map()
    return params


def get_random_geometric_augmentation_params():
    return dict(hflip=random.choice([True, False]), n_rot90=random.choice([0, 1, 2, 3]))


def get_random_color_map(change_background_probability=0.1):
    colors = list(range(10))
    if random.random() < change_background_probability:
        new_colors = list(range(10))
        random.shuffle(new_colors)
    else:
        new_colors = list(range(1, 10))
        random.shuffle(new_colors)
        new_colors = [0] + new_colors

    color_map = {x: y for x, y in zip(colors, new_colors)}
    return color_map

#### Experiment

In [ ]:
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
print(f"Generating {n_preds} predictions for {len(task_ids)} tasks...")
for epoch in tqdm(range(n_preds), smoothing=0, desc="Generating predictions"):
    prompts, data_augmentation_params = [], []
    for task_id in task_ids:
        params = get_random_data_augmentation_params()
        data_augmentation_params.append(params)
        task = get_task(task_id)
        task = apply_data_augmentation(task, **params)
        prompt = create_prompt_from_task(
            task, grid_encoder=grid_encoder, tokenizer=tokenizer, shuffle_train_samples=True)
        prompts.append(prompt)
    # pretty_print_prompt(prompts[0], default_color='white')

    outputs = llm.generate(prompts, sampling_params)
    for task_id, responses, params in zip(task_ids, outputs, data_augmentation_params):
        task = get_task(task_id)
        task = apply_data_augmentation(task, **params)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                predicted_code[task_id].append(code)
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    task_predicted_outputs = [revert_data_augmentation(output, **params) for output in task_predicted_outputs]
                    predicted_outputs[task_id].append(task_predicted_outputs)
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

### Remove one training sample (constraint relaxation)

In [ ]:
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
print(f"Generating {n_preds} predictions for {len(task_ids)} tasks...")
for epoch in tqdm(range(n_preds), smoothing=0, desc="Generating predictions"):
    prompts = []
    for task_id in task_ids:
        prompt = create_prompt_from_task(
            get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer,
            shuffle_train_samples=True, remove_last_train_sample=epoch != 0)
        prompts.append(prompt)
    # pretty_print_prompt(prompts[0], default_color='white')

    outputs = llm.generate(prompts, sampling_params)
    for task_id, responses in zip(task_ids, outputs):
        task = get_task(task_id)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                predicted_code[task_id].append(code)
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    predicted_outputs[task_id].append(task_predicted_outputs)
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {type(e)} {e}")

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

### Multiple functions per prediction

Maybe if I ask to generate multiple functions on the same prompt, the model will have the intelligence to take different approaches.

#### Code

In [ ]:
prompt_multiple_functions_template_text = """You are tasked with solving a transformation problem from the Abstraction and Reasoning Challenge (ARC).
Implement the transformation rules as a Python function.
You should only write the implemented the transformation in code.
You must write code in triple backticks (```python and then ). You must write a function called `transform` which takes a single argument, the input grid as `list[list[int]]`, and returns the transformed grid (also as `list[list[int]]`).

## Key Priors:

- **Objectness**: Consider the grid as containing objects (groups of connected cells) rather than just individual pixels.
- **Goal-Directed**: The transformation should achieve a specific goal, such as creating symmetry or changing the color of specific objects.
- **Numbers & Counting**: Keep track of the number of objects, sizes, and their relative positions.
- **Geometry & Topology**: Use spatial relationships such as adjacency, enclosure, or symmetry.

Carefully analyze the examples and find the underlying transformation logic.

## Domain Specific Primitive Functions

You can use the already implemented following functions to manipulate the grid:

{{ dsl }}

The dsl has been already imported, so just simply call the functions as needed. F.e. dsl.foo()
Do not import the dsl again, just use it directly.

## Examples

Below are several input-output examples that illustrate the transformation.
Your function should generalize the pattern from these examples to solve any input following the same logic.

{% for sample in train_samples %}
### Example {{ loop.index }}

#### Input

{{ sample.input }}

#### Output

{{ sample.output }}
{% endfor %}

## Code

Implement the transformation rules as a Python function.
Please write {{ n }} distinct implementations of the `transform` function, each solving the same problem in a different way.
Place each implementation in a separate code block, and ensure that each implementation is unique.
"""
prompt_multiple_functions_template = Template(prompt_multiple_functions_template_text)

def create_prompt_requesting_multiple_functions_from_task(
        n_output_functions, task, grid_encoder, tokenizer, shuffle_train_samples=False, remove_last_train_sample=False):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    if shuffle_train_samples:
        random.shuffle(train_samples)
    if remove_last_train_sample and len(train_samples) > 1:
        train_samples = train_samples[:-1]
    render_kwargs = dict(train_samples=train_samples, n=n_output_functions,
                         dsl=extract_footprint('arc25.BARC_dsl', show_types=True))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_multiple_functions_template.render(**render_kwargs)}]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

In [ ]:
def parse_multiple_python_code_snippets(text):
    # Extract Python code from the text
    if '```python' not in text:
        return
    for code in text.split('```python')[1:]:
        if '```' in code:
            code = code.split('```')[0].strip()
            if code:
                yield code

#### Experiments

In [ ]:
n_functions_per_prediction = 8
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=8192)
predictions = {key: [] for key in task_ids}
predicted_code = {key: [] for key in task_ids}
predicted_outputs = {key: [] for key in task_ids}
t0 = time.time()
output_tokens = []
print(f"Generating {n_preds} predictions for {len(task_ids)} tasks...")
for epoch in tqdm(range(n_preds//n_functions_per_prediction), smoothing=0, desc="Generating predictions"):
    prompts = []
    for task_id in task_ids:
        prompt = create_prompt_requesting_multiple_functions_from_task(
            n_output_functions = n_functions_per_prediction,
            task=get_task(task_id), grid_encoder=grid_encoder, tokenizer=tokenizer,
            shuffle_train_samples=True, remove_last_train_sample=False)
        prompts.append(prompt)
    # pretty_print_prompt(prompts[0], default_color='white')

    outputs = llm.generate(prompts, sampling_params)
    for task_id, responses in zip(task_ids, outputs):
        task = get_task(task_id)
        output_tokens.append(len(responses.outputs[0].token_ids))
        predictions[task_id].append(responses.outputs[0].text)
        for code in parse_multiple_python_code_snippets(responses.outputs[0].text):
            predicted_code[task_id].append(code)
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                predicted_outputs[task_id].append(task_predicted_outputs)
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}: {type(e)} {e}")
    print([len(codes) for codes in predicted_code.values()])

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
print(f'Max tokens per output: {max(output_tokens)}')
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

In [ ]:
for prediction in predictions[task_ids[np.argmin([len(codes) for codes in predicted_code.values()])]]:
    print(prediction)
    print('-' * 80)

In [ ]:
len(outputs[0].outputs[0].token_ids)

In [ ]:
predicted_code['007bbfb7']

In [ ]:
print(outputs[0].outputs[0].text)

## Solution refinement

### Default Configuration

In [ ]:
model_path = "/home/gbarbadillo/models/Qwen2.5-Coder-7B-Instruct"
llm, tokenizer = load_model(model_path, use_4bit_quantization=False, tensor_parallel_size=1)

In [ ]:
n_preds = 16
task_ids = list(training_challenges.keys())
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
grid_encoder = create_grid_encoder('GridShapeEncoder(RowNumberEncoder(MinimalGridEncoder()))')

### Baseline

In [ ]:
Prediction = namedtuple("Prediction", ['code', 'outputs', 'exception', 'scores'])

def format_exception(e):
    message = str(e).replace('arc25.BARC_dsl', 'dsl')
    return f"{e.__class__.__name__}: {message}"

In [ ]:
sampling_params = SamplingParams(n=n_preds, temperature=1.0, top_p=0.95, max_tokens=2048)
predictions = {key: [] for key in task_ids}
t0 = time.time()
prompts = []
for task_id in task_ids:
    prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder,
                                        tokenizer=tokenizer, shuffle_train_samples=False)
    prompts.append(prompt)

outputs = llm.generate(prompts, sampling_params)

for task_id, responses in zip(task_ids, outputs):
    task = get_task(task_id)
    for i, output in enumerate(responses.outputs):
        code = parse_python_code(output.text)
        if code:
            code = curate_python_code(code)
            try:
                task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                task_predicted_outputs = validate_outputs(task_predicted_outputs)
                prediction = Prediction(
                    code=code, outputs=task_predicted_outputs, exception=None,
                    scores=[pixel_similarity_score(output, pred) for output, pred in zip(task.outputs, task_predicted_outputs)])
            except Exception as e:
                logging.error(f"Error executing code for task {task_id}, response {i}: {format_exception(e)}")
                prediction = Prediction(code=code, outputs=None, exception=format_exception(e),
                                        scores=None)
            predictions[task_id].append(prediction)

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
max_output_tokens = max(max(len(output.token_ids) for output in responses.outputs) for responses in outputs)
print(f'Max output tokens: {max_output_tokens}')
predicted_code = {key: [pred.code for pred in values] for key, values in predictions.items()}
predicted_outputs = {key: [pred.outputs for pred in values if pred.outputs is not None] for key, values in predictions.items()}
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds)
df

In [ ]:
successful_execution_feedback_template_text = """## Execution feedback

Please refine the code to ensure it produces the expected outputs for all examples.

### Execution outputs

These are the outputs of the code execution for each example. Compare them with the expected
outputs and refine the code if necessary to achieve the desired transformation.
{% for output in outputs %}
#### Example {{ loop.index }} execution output

{{ output }}
{% endfor %}

### Metrics

These are the pixel similarity scores for each sample: {{ scores}}
All scores need to be 1.0 to consider the code correct.
"""
successful_execution_feedback_template = Template(successful_execution_feedback_template_text)

failed_execution_feedback_template_text = """## Execution feedback

Execution of the code failed with the following exception:
```
{{ exception }}
```

Please refine the code to ensure it executes successfully and produces the expected outputs for all examples.
"""
failed_execution_feedback_template = Template(failed_execution_feedback_template_text)

def generate_execution_feedback_message(prediction, grid_encoder=grid_encoder):
    if prediction.outputs is None:
        return failed_execution_feedback_template.render(exception=prediction.exception)
    outputs = [grid_encoder.to_text(output) for output in prediction.outputs]
    scores = np.array(prediction.scores).round(2).tolist()
    feedback_message = successful_execution_feedback_template.render(outputs=outputs, scores=scores)
    return feedback_message

In [ ]:
def create_refine_prompt_from_task(task, prediction, grid_encoder, tokenizer, shuffle_train_samples=False, remove_last_train_sample=False):
    train_samples = [{'input': grid_encoder.to_text(grid), 'output': grid_encoder.to_text(output)} for grid, output in zip(task.inputs, task.outputs)]
    if shuffle_train_samples:
        random.shuffle(train_samples)
    if remove_last_train_sample and len(train_samples) > 1:
        train_samples = train_samples[:-1]
    render_kwargs = dict(train_samples=train_samples, dsl=extract_footprint('arc25.BARC_dsl', show_types=True))
    messages = [{"role": "system", "content": system_prompt},
                {"role": "user", "content": prompt_template.render(**render_kwargs)},
                {'role': 'assistant', 'content': f'```python\n{prediction.code}\n```'},
                {'role': 'user', 'content': generate_execution_feedback_message(prediction, grid_encoder=grid_encoder)},
               ]
    prompt = tokenizer.apply_chat_template(messages,
                                            tokenize=False,
                                            add_generation_prompt=True,
                                            # enable_thinking=False,
                                            )
    return prompt

In [ ]:
sampling_params = SamplingParams(n=1, temperature=1.0, top_p=0.95, max_tokens=2048)
t0 = time.time()

for epoch in tqdm(range(n_preds), smoothing=0):
    print(f"Generating {n_preds} predictions for {len(task_ids)} tasks, epoch {epoch + 1}/{n_preds}...")
    prompts = []
    for task_id in task_ids:
        if predictions[task_id]:
            # prediction = random.choice(predictions[task_id]) # random choose
            prediction = predictions[task_id][epoch % len(predictions[task_id])]  # loop over the predictions
            prompt = create_refine_prompt_from_task(get_task(task_id), prediction, grid_encoder=grid_encoder,
                                                    tokenizer=tokenizer, shuffle_train_samples=False)
        else:
            # If no previous prediction, use the original task prompt
            logging.warning(f"No previous prediction for task {task_id}, using original task prompt.")
            prompt = create_prompt_from_task(get_task(task_id), grid_encoder=grid_encoder,
                                                tokenizer=tokenizer, shuffle_train_samples=False)
        prompts.append(prompt)

    outputs = llm.generate(prompts, sampling_params)
    max_output_tokens = max(max(len(output.token_ids) for output in responses.outputs) for responses in outputs)
    print(f'Max output tokens: {max_output_tokens}')
    for task_id, responses in zip(task_ids, outputs):
        task = get_task(task_id)
        for i, output in enumerate(responses.outputs):
            code = parse_python_code(output.text)
            if code:
                code = curate_python_code(code)
                try:
                    task_predicted_outputs = safe_code_execution(add_additional_imports(code), task.inputs, func_name='transform', dsl=dsl)
                    task_predicted_outputs = validate_outputs(task_predicted_outputs)
                    prediction = Prediction(
                        code=code, outputs=task_predicted_outputs, exception=None,
                        scores=[pixel_similarity_score(output, pred) for output, pred in zip(task.outputs, task_predicted_outputs)])
                except Exception as e:
                    logging.error(f"Error executing code for task {task_id}, response {i}: {format_exception(e)}")
                    prediction = Prediction(code=code, outputs=None, exception=format_exception(e),
                                            scores=None)
                predictions[task_id].append(prediction)

inference_time = time.time() - t0
print(f'Inference time: {inference_time:.1f} seconds')
predicted_code = {key: [pred.code for pred in values] for key, values in predictions.items()}
predicted_outputs = {key: [pred.outputs for pred in values if pred.outputs is not None] for key, values in predictions.items()}
df = compute_search_metrics(task_ids, predicted_code, predicted_outputs, n_preds*2)
df

In [ ]:
for task_id, preds in predictions.items():
    for pred in preds:
        if pred.outputs is not None:
            try:
                validate_outputs(pred.outputs)
            except Exception as e:
                logging.error(f"Validation error for task {task_id}: {format_exception(e)}")
                continue

In [ ]:
for prompt in prompts:
    if len(tokenizer.tokenize(prompt)) > 32000:
        pretty_print_prompt(prompt, default_color='white')

In [ ]:
len(predictions['007bbfb7'])

In [ ]:
pretty_print_prompt(prompts[0], default_color='white')

In [ ]:
outputs[0].outputs[0].text

- I would like to see the distribution of output tokens.
- Problems with `arc25.code_execution.TimeoutException: Code execution exceeded time limit!`
- `Token indices sequence length is longer than the specified maximum sequence length for this model (34076 > 32768). Running this sequence through the model will result in indexing errors`

## TODO

- [x] Create a prompt with the available DSL functions and the training ARC task
- [x] Fix VLLM initialization issues with proper memory management
- [x] Verify the effect of caching
- [x] Generate some code that can be used to test the new BARC dsl
- [x] Update the library to be able to select which DSL to use when executing code
- [x] Verify that I can execute the code generated with the BARC dsl
- [x] Add security checks to code, I have seen some input required in the code
- [ ] Try to solve some easy task with independent sampling
  - [x] How frequently is the dsl used?
  - [x] Influence of the model
  - [x] Implement output validation, and simplify the metric. Those are different responsabilities
  - [x] Correct grids
  - [x] Unique outputs
  - [x] Everything into a dataframe
  - [ ] Metrics distribution
  - [ ] Visualization of the predictions
  - [ ] Global metrics vs task specific analysis
- [ ] Create a refine prompt
- [ ] Make a more complex tree search
- [x] Validate the tokenizer on the input grids

```python
for i in range(10):
    print(i, {word for word in tokenizer.vocab if str(i) in word})
```